In [1]:
#################################################################
##
##  Code to turn river line strings from HydroSheds into bianary raster
##  in the same resolution and crs (EPSG:54009) at 1km res as other data.
##  Next it takes the ocean-land mask to merge it with the
##  coastline
##
##  Built with conda env geo37
##
##  By Cascade Tuholske 2019-10-16
##
#################################################################

In [2]:
# Dependencies 
##############################################################################################################
from glob import glob
import numpy as np
import rasterio 
from rasterio import features
import pandas as pd
import geopandas as gpd
import os
from skimage.morphology import erosion

In [3]:
# Files and File Paths
##############################################################################################################
DATA_IN = '../../data/'


In [4]:
#### Functions
##############################################################################################################

def burn_rst(rst_in, polys, fn_out):
    """ Function burns polygons into a bianary raster, polys must have a value column labeled 'true'
    Args:
        rst_in = file path to raster for template
        polys = polygons to burn int, values column must be labeled 'true'
        fn_out = file path for burned raster
    """
    
    # Get array as template
    rst = rasterio.open(rst_in)
    out_arr = rst.read(1) # get an array to burn shapes
    out_arr.fill(0) # revalue rst to an Nan Value before burning in polygons
    out_arr = out_arr.astype('uint8') # change dtype for file size
    
    # Update Meta Data
    meta = rst.meta
    meta['dtype'] = "uint8"
    meta['nodata'] = 99 # NA as 99
    
    # extract geom and values to burn
    shapes = ((geom,value) for geom, value in zip(polys['geometry'], polys['true']))

    # burn shapes intp an array
    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=rst.transform, all_touched=True)

    # write our raster to disk
    with rasterio.open(fn_out, 'w', **meta) as out:
        out.write_band(1, burned)

    print(fn_out + "  saved!")

In [5]:
# Make Coastline Rst
##############################################################################################################
rst_in = DATA_IN+'interim/oceanland_mask.tif'
rst = rasterio.open(rst_in)


In [6]:
# erode by 1-km 
arr = rst.read(1)
arr_eroded = erosion(arr) 

In [7]:
# add them together and revalue
coast_arr = arr+arr_eroded

In [9]:
# 0 = ocean, 1 = coastline, 2 = land
coast_arr[coast_arr == 2] = 0

In [10]:
# write it out
meta = rst.meta
crs = rst.crs # get CRS for later 
rst_out = DATA_IN+'interim/coastline.tif'
with rasterio.open(rst_out, 'w', **meta) as dst:
    dst.write_band(1, coast_arr)
print('coastline rst done')

In [12]:
# Stack Hydrosheds & Burn as a Raster
##############################################################################################################
shp_out = DATA_IN+'interim/Hydro30s_rivers_stacked.shp'
dir_list = glob(DATA_IN+'raw/Hydrosheds/RIV_30s/*/')

# Open all hydro shed polys and stack them for later
gpd_out = gpd.GeoDataFrame()

for dir_nm in dir_list:
    for fn in os.listdir(dir_nm):
        if fn.endswith('.shp'): 
            river = gpd.read_file(dir_nm+fn) # open shape file
            gpd_out = gpd_out.append(river) # merge to gpd_out
            print(gpd_out.shape)

# write out to file            
gpd_out.to_file(shp_out)
print('rivers stack saved')

(76685, 3)
(793547, 3)
(1232703, 3)
(1696879, 3)
(2566735, 3)
(2999273, 3)
(3199865, 3)
rivers stack saved


In [13]:
# add column for burning
gpd_out['true'] = 1
gpd_out_crs = gpd_out.to_crs(rst.crs)

In [15]:
# Burn River Raster 
rst_out = DATA_IN+'interim/river_rst.tif'
burn_rst(rst_in = rst_in, polys = gpd_out_crs , fn_out = rst_out)
print('river rst done')

../../data/interim/river_rst.tif  saved!


In [18]:
# merge river rst and coastline rst 
##############################################################################################################
riv_arr = rasterio.open(rst_out).read(1)
coast_riv = riv_arr+coast_arr

rst_out = DATA_IN+'interim/riv_30s_coastlines.tif'
with rasterio.open(rst_out, 'w', **meta) as dst:
    dst.write_band(1, coast_riv)
print('River-coastline rst is done - ALL DONE')

NameError: name 'coast_arr' is not defined